In [9]:
pip install geopandas shapely folium osmnx matplotlib folium streamlit-folium ipython streamlit pyngrok


In [12]:
# Save the Streamlit code to a file
streamlit_script = """
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import folium
import streamlit as st
from streamlit_folium import folium_static  # Import folium_static
import requests

# User input for the city name
city_name = st.text_input("Enter the city name:")

if city_name:
    # Download city boundary and green spaces
    st.write("Downloading data...")
    try:
        city_boundary = ox.geocode_to_gdf(city_name)
        green_spaces = ox.features_from_place(city_name, tags={"leisure": "park"})
    except Exception as e:
        st.write(f"Error downloading data: {e}")
        st.stop()

    # Reproject to a projected CRS for buffer analysis
    projected_crs = "EPSG:32651"  # Adjusted for UTM Zone 51N (Philippines)
    city_boundary = city_boundary.to_crs(projected_crs)
    green_spaces = green_spaces.to_crs(projected_crs)

    # Buffer analysis (500m radius)
    st.write("Performing buffer analysis...")
    green_buffers = green_spaces.buffer(500)  # 500 meters in projected CRS

    # Check for empty or invalid geometries
    green_buffers = green_buffers[green_buffers.is_valid]

    # Reproject back to WGS84 for visualization
    city_boundary = city_boundary.to_crs(epsg=4326)

    # Convert green_buffers back to GeoDataFrame
    green_buffers_gdf = gpd.GeoDataFrame(geometry=green_buffers, crs=projected_crs)
    green_buffers_gdf = green_buffers_gdf.to_crs(epsg=4326)

    # Clip the green buffers to the city boundary to ensure no buffer goes outside the city
    green_buffers_gdf = gpd.clip(green_buffers_gdf, city_boundary)

    # Retrieve actual population data (replace with a valid source if needed)
    def get_population_data(city_boundary):
        st.write("Fetching population data...")
        bounds = city_boundary.total_bounds
        api_url = (
            f"https://api.worldpop.org/v1/wfs?service=WFS&request=GetFeature&"
            f"bbox={bounds[1]},{bounds[0]},{bounds[3]},{bounds[2]}&outputFormat=application/json"
        )
        response = requests.get(api_url)
        if response.status_code == 200:
            population_data = gpd.read_file(response.text)
            return population_data
        else:
            st.write("Failed to retrieve population data.")
            return gpd.GeoDataFrame()

    population_points = get_population_data(city_boundary)

    # Ensure CRS compatibility
    if not population_points.empty:
        population_points = population_points.to_crs(city_boundary.crs)

    # Spatial join to find population with green space access
    population_points["has_access"] = population_points.geometry.apply(
        lambda x: any(green_buffers_gdf.contains(x))
    )
    underserved_population = population_points[~population_points["has_access"]]

    # Visualization using Folium
    st.write("Creating interactive map...")
    m = folium.Map(location=[city_boundary.geometry.centroid.y.mean(), city_boundary.geometry.centroid.x.mean()], zoom_start=14)

    # Add green spaces (buffers) to the map
    for _, row in green_buffers_gdf.iterrows():
        folium.GeoJson(
            row['geometry'],
            style_function=lambda x: {'fillColor': 'green', 'color': 'green', 'fillOpacity': 0.4}
        ).add_to(m)

    # Add population points to the map
    for _, row in population_points.iterrows():
        folium.CircleMarker(
            location=(row.geometry.y, row.geometry.x),
            radius=5, color="blue", fill=True, fill_opacity=0.7,
            popup=f"Population: {row['population']}"
        ).add_to(m)

    # Add underserved population points to the map
    for _, row in underserved_population.iterrows():
        folium.CircleMarker(
            location=(row.geometry.y, row.geometry.x),
            radius=5, color="red", fill=True, fill_opacity=0.7,
            popup=f"Underserved Population: {row['population']}"
        ).add_to(m)

    # Display map in Streamlit
    folium_static(m)  # Use folium_static to display the map

"""
# Write the script to a Python file
with open('streamlit_app.py', 'w') as f:
    f.write(streamlit_script)


In [13]:
# Import required libraries
from pyngrok import ngrok
import os

# Set ngrok auth token (replace with your actual ngrok auth token)
ngrok.set_auth_token("2qx3qqIK8jFwS2FfXQ3UnBpAzy2_7a6W7JNihXrSiR6jB3Hu3")

# Kill any existing ngrok tunnels
ngrok.kill()

# Start ngrok tunnel for Streamlit on port 8501
public_url = ngrok.connect(8501)
print(f"Streamlit app is running at: {public_url}")

# Run Streamlit app in the background (replace with your actual Streamlit file name)
os.system("streamlit run streamlit_app.py &")


Streamlit app is running at: NgrokTunnel: "https://45b2-34-80-192-249.ngrok-free.app" -> "http://localhost:8501"


0